In [1]:
!pip install tensorflow_hub


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

# MoveNet 모델 로드 (저장 없이 직접 사용)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))  # Lightning 모델은 192x192 입력
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # (17, 3)
    return keypoints


In [3]:
!pip install opencv-python

In [4]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import requests
import threading
import torch
import torch.nn as nn
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

# --- 1. 낙상 분류 모델 정의 및 로드 ---
class FallClassifierWithAttention(nn.Module):
    def __init__(self, input_dim=51, hidden_dim=256, num_layers=3, output_dim=3, dropout=0.5):
        super(FallClassifierWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_scores = self.attention(lstm_out)
        attn_weights = torch.softmax(attn_scores, dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        out = self.dropout(context)
        out = self.fc(out)
        return out

# 모델 초기화 및 가중치 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FallClassifierWithAttention().to(device)
model.load_state_dict(torch.load("../model/best_model.pt", map_location=device))
model.eval()

# --- 2. FastAPI 서버 설정 ---
app = FastAPI()

# CORS 설정 (로컬 개발용)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

class KeypointSequence(BaseModel):
    sequence: list  # (100, 51) 2차원 리스트

@app.post("/predict")
async def predict_pose(data: KeypointSequence):
    x = np.array(data.sequence, dtype=np.float32)
    x = torch.tensor(x).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(x)
        pred = torch.argmax(output, dim=1).item()
    return {"result": pred}

# 서버 실행 함수
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001)

# --- 3. MoveNet 및 웹캠 처리 ---
# MoveNet 모델 로드
model_movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model_movenet.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))
    img = img.astype(np.int32)
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img, dtype=tf.int32)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]
    keypoints_xyc = np.stack([
        keypoints[:, 1],  # x (0~1)
        keypoints[:, 0],  # y (0~1)
        keypoints[:, 2]   # confidence
    ], axis=1).flatten()
    return keypoints_xyc

# --- 4. 메인 실행 흐름 ---
if __name__ == "__main__":
    # 서버를 별도 스레드에서 실행
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    print("FastAPI 서버가 백그라운드에서 시작되었습니다 (http://localhost:8001)")

    # 웹캠 처리 시작
    cap = cv2.VideoCapture(0)
    sequence = []

    while True:
        ret, frame = cap.read()
        if not ret:
            print("웹캠 프레임을 읽을 수 없습니다.")
            break

        try:
            keypoints = get_keypoints(frame)
            sequence.append(keypoints.tolist())

            if len(sequence) == 100:
                data = {"sequence": sequence}
                try:
                    # 로컬 서버로 요청 전송
                    res = requests.post("http://localhost:8001/predict", json=data, timeout=3)
                    result = res.json()["result"]
                    status = ["Normal", "Suspicious", "Fall"][result]
                    print(f"예측 결과: {status}")
                    
                    # Spring Boot로 전송 (옵션)
                    requests.post("http://localhost:8080/fall", json={"status": status})
                except Exception as e:
                    print(f"추론 서버 연결 실패: {e}")
                sequence = []
        except Exception as e:
            print(f"관절 추출 에러: {e}")

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 종료
            break

    cap.release()
    cv2.destroyAllWindows()


FastAPI 서버가 백그라운드에서 시작되었습니다 (http://localhost:8001)


INFO:     Started server process [24540]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:9877 - "POST /predict HTTP/1.1" 200 OK
예측 결과: Suspicious
INFO:     127.0.0.1:9890 - "POST /predict HTTP/1.1" 200 OK
예측 결과: Suspicious


KeyboardInterrupt: 